In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os


In [3]:
%pwd

'c:\\Users\\Akhil\\OneDrive\\Desktop\\animal-disease-classification-with-llm\\research'

In [4]:
os.chdir(r"\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm")

In [5]:
%pwd

'c:\\Users\\Akhil\\OneDrive\\Desktop\\animal-disease-classification-with-llm'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
 

In [7]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    # defining the dataingestion config as the return type
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config
    
    

In [9]:
import os
import urllib.request as request
import zipfile
from classifier import logger
import gdown
from classifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) -> Path:
        if not os.path.exists(self.config.local_data_file):

            url = self.config.source_URL

            # if it is a google drive link, download using gdown
            if "drive.google.com" in url:
                gdown.download(url, str(self.config.local_data_file), quiet=False)
                logger.info(f"Downloaded from Google Drive: {self.config.local_data_file}")
            else:
                filename, headers = request.urlretrieve(
                    url, self.config.local_data_file
                )
                logger.info(f"File: {filename} downloaded with info: \n{headers}")

        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )

        return self.config.local_data_file
    
    def extract_zip_file(self) -> None:
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# Now we create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-20 11:06:34,044 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2026-01-20 11:06:34,057 - INFO - common - yaml file: params.yaml loaded successfully]
created directory at: artifacts
created directory at: artifacts/data_ingestion


Downloading...
From (original): https://drive.google.com/uc?export=download&id=17IkxLdZglyS3OsnTqi_QS8TC0V9oN904
From (redirected): https://drive.google.com/uc?export=download&id=17IkxLdZglyS3OsnTqi_QS8TC0V9oN904&confirm=t&uuid=530eaf16-d2c4-4a96-adb9-11840f614443
To: c:\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm\artifacts\data_ingestion\data.zip
100%|██████████| 248M/248M [00:30<00:00, 8.17MB/s] 

[2026-01-20 11:07:07,045 - INFO - 465068388 - Downloaded from Google Drive: artifacts\data_ingestion\data.zip]
